### Upload time period mapping to MDB

In [52]:
import requests
import importlib
import blizzard_api
import mplusdb
import blizzard_credentials
import pandas as pd
import utils
import mplusdb

In [6]:
#grab access token
importlib.reload(blizzard_credentials)
tokens_file_path = '.api_tokens'
authorization = blizzard_credentials.Credentials(tokens_file_path)
access_token = authorization.access_token

In [56]:
# get period ids and end/start timestamps for every region
parser = blizzard_api.ResponseParser()
period_timestamps = []
for region in ['us', 'eu', 'tw', 'kr']:
    region_code = utils.Utils().encode_region(region)
    call_factory = blizzard_api.CallFactory(
        region = region, access_token = access_token)
    # get list of periods on record - should be the same for every region
    period_index_url = call_factory.get_timeperiod_index_url()
    response = requests.get(period_index_url)
    periods = parser.parse_timeperiod_index_json(response.json())
    
    # get start/end timestamp for every period
    for period in periods:
        period_url = call_factory.get_timeperiod_url(period)
        response = requests.get(period_url)
        start, end = parser.parse_timeperiod_json(response.json())
        period_timestamps.append((region_code, period, start, end))
        print('\r region %s period %s' % (region, period), end='')
print('')

 region kr period 762


In [59]:
pd.DataFrame(period_timestamps).head(5)

,0,1,2,3
0,1,641,1523372400000,1523977199000
1,1,642,1523977200000,1524581999000
2,1,643,1524582000000,1525186799000
3,1,644,1525186800000,1525791599000
4,1,645,1525791600000,1526396399000


In [75]:
importlib.reload(mplusdb)
mdb = mplusdb.MplusDatabase('.db_config')

In [76]:
mdb.insert(table ='period',
           data = period_timestamps)